In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, "/home/gy46/")

In [2]:
import numpy as np
import os.path as osp

from latent_3d_points.src.evaluation_metrics import minimum_mathing_distance, \
jsd_between_point_cloud_sets, coverage

from latent_3d_points.src.in_out import snc_category_to_synth_id,\
                                        load_all_point_clouds_under_folder

In [16]:
# top_in_dir = '../data/shape_net_core_uniform_samples_2048/' # Top-dir of where point-clouds are stored.
# top_in_dir = '../data/ShapeNetV1PCOutput/' # Top-dir of where point-clouds are stored.
top_in_dir = '../data/ShapeNetCore.v2.PC15k/'
class_name = raw_input('Give me the class name (e.g. "chair"): ').lower()
syn_id = snc_category_to_synth_id()[class_name]
class_dir = osp.join(top_in_dir , syn_id, 'val')
normalize_shape=True
# all_pc_data = load_all_point_clouds_under_folder(class_dir, n_threads=8, file_ending='.ply', verbose=True)
all_pc_data = load_all_point_clouds_under_folder(
    class_dir, n_threads=8, file_ending='.npy', verbose=True, normalize=normalize_shape)

Give me the class name (e.g. "chair"): chair
662 pclouds were loaded. They belong in 1 shape-classes.


In [17]:
# n_ref = 100 # size of ref_pcs.
# n_sam = 150 # size of sample_pcs.
all_ids = np.arange(all_pc_data.num_examples)
print(len(all_ids))
ref_ids = np.random.choice(all_ids, len(all_ids), replace=False)
sam_ids = np.random.choice(all_ids, len(all_ids), replace=False)

ref_pcs = all_pc_data.point_clouds[ref_ids][:,10000:12048,:]
sample_pcs = all_pc_data.point_clouds[sam_ids][:,12048:14096,:]

662


In [18]:
 all_pc_data.point_clouds.shape

(662, 15000, 3)

In [19]:
ref_pcs.shape, sample_pcs.shape

((662, 2048, 3), (662, 2048, 3))

In [20]:
ae_loss = 'chamfer'  # Which distance to use for the matchings.

if ae_loss == 'emd':
    use_EMD = True
else:
    use_EMD = False  # Will use Chamfer instead.
    
batch_size = 100     # Find appropriate number that fits in GPU.
normalize = False     # Matched distances are divided by the number of 
                      # points of thepoint-clouds.
class_name

'chair'

In [21]:
from latent_3d_points.src.evaluation_metrics_fast import MMD_COV_EMD_CD
mmd_emd, mmd_cd, cov_emd, cov_cd = MMD_COV_EMD_CD(
    sample_pcs, ref_pcs, 32, normalize=normalize, verbose=True)


MMD-COV Loop: 100%|██████████| 662/662 [40:05<00:00,  3.63s/it]


In [24]:

print("MMD-EMD:%s"%mmd_emd)
print("MMD-CD:%s"%mmd_cd)
print("COV-EMD:%s"%cov_emd)
print("COV-CD:%s"%cov_cd)


MMD-EMD:387.56354
MMD-CD:24.056911
COV-EMD:0.987915407855
COV-CD:0.996978851964


In [26]:
class_name, normalize_shape

('chair', True)